# Imports

In [1]:
import os
import sys
import time
import pprint

from eth_account import Account
from eth_account.signers.local import LocalAccount
import web3
from web3 import Web3, EthereumTesterProvider
from web3.middleware import construct_sign_and_send_raw_middleware
import solcx

# Set-up Alchemy & Accounts

In [2]:
infura_key = os.environ["INFURA_KEY"]
infura_url = f"https://goerli.infura.io/v3/{infura_key}"
w3 = Web3(Web3.HTTPProvider(infura_url))

print(w3.isConnected())

True


In [3]:
account_addr = "0x6531e2613bbbEEcd898356F7b9bEfBaEfd42804B"
private_key = os.environ["PRIVATE_KEY"]

account: LocalAccount = Account.from_key(private_key)
w3.middleware_onion.add(construct_sign_and_send_raw_middleware(account))

print(f"Your hot wallet address is {account.address}")

Your hot wallet address is 0x6531e2613bbbEEcd898356F7b9bEfBaEfd42804B


# Functions

In [5]:
def compile_source_file(file_path):
    with open(file_path, 'r') as f:
        source = f.read()
    return solcx.compile_source(source, output_values=["abi", "bin", "bin-runtime"])

# Compile contract

In [6]:
compiled = compile_source_file("contracts/PriceOracle.sol")
contract_interface = compiled["<stdin>:PriceOracle"]

# LINK/ETH

In [7]:
# Address to LINK/ETH oracle in Goerli network.
priceFeedAddress = "0xb4c4a493AB6356497713A78FFA6c60FB53517c63"

func=w3.eth.contract(
    abi=contract_interface["abi"],
    bytecode=contract_interface["bin"]).constructor(priceFeedAddress)

tx_hash = func.transact({"from": account_addr})

In [ ]:
address = w3.eth.get_transaction_receipt(tx_hash)["contractAddress"]
contract = w3.eth.contract(address=address, abi=contract_interface["abi"])

# Get price of LINK token in wei.
link_wei = contract.functions.getLatestPrice().call()

In [18]:
# Get price in ETH
link_wei/10**18

0.004056058300937638

# BTC/ETH

In [24]:
# Address to BTC/ETH oracle in Goerli network.
priceFeedAddress = "0x779877A7B0D9E8603169DdbD7836e478b4624789"

func=w3.eth.contract(
    abi=contract_interface["abi"],
    bytecode=contract_interface["bin"]).constructor(priceFeedAddress)

tx_hash = func.transact({"from": account_addr})

In [27]:
address = w3.eth.get_transaction_receipt(tx_hash)["contractAddress"]
contract = w3.eth.contract(address=address, abi=contract_interface["abi"])

# Get price of BTC in wei.
btc_eth = contract.functions.getLatestPrice().call()

In [28]:
# Get price in ETH
btc_eth/10**18

14.853088403199164

In [20]:
btc_eth

4097403941144435

# BTC/USD

In [29]:
# Address to BTC/USD oracle in Goerli network.
priceFeedAddress = "0xA39434A63A52E749F02807ae27335515BA4b07F7"

func=w3.eth.contract(
    abi=contract_interface["abi"],
    bytecode=contract_interface["bin"]).constructor(priceFeedAddress)

tx_hash = func.transact({"from": account_addr})

In [30]:
address = w3.eth.get_transaction_receipt(tx_hash)["contractAddress"]
contract = w3.eth.contract(address=address, abi=contract_interface["abi"])

# Get price of BTC in USD.
btc_usd = contract.functions.getLatestPrice().call()

In [34]:
# Get price in USD.
# Price typically scaled by 1e8 (or 10^8) to accommodate decimal values.
btc_usd / 100000000

25886.9